<a href="https://colab.research.google.com/github/PavelStelmakhV/hw310-keras-fasion-mnist/blob/main/keras_fasion_mnist_hw.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf

import numpy as np
import matplotlib.pyplot as plt

2.14.0


In [3]:
fashion_mnist = tf.keras.datasets.fashion_mnist

(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

4422102/4422102 [==============================] - 0s 0us/step


In [4]:
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

In [7]:
train_images = train_images / 255.0

test_images = test_images / 255.0

In [9]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10)
])

In [10]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [11]:
model.fit(train_images, train_labels, epochs=10)

Epoch 1/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.4990 - accuracy: 0.8245
Epoch 2/10
1875/1875 [==============================] - 9s 5ms/step - loss: 0.3730 - accuracy: 0.8650
Epoch 3/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.3346 - accuracy: 0.8773
Epoch 4/10
1875/1875 [==============================] - 9s 5ms/step - loss: 0.3137 - accuracy: 0.8850
Epoch 5/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.2957 - accuracy: 0.8907
Epoch 6/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.2803 - accuracy: 0.8957
Epoch 7/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.2674 - accuracy: 0.9003
Epoch 8/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.2580 - accuracy: 0.9045
Epoch 9/10
1875/1875 [==============================] - 9s 5ms/step - loss: 0.2493 - accuracy: 0.9065
Epoch 10/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.240

In [12]:
test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)

print('\nTest accuracy:', test_acc)

313/313 - 1s - loss: 0.3362 - accuracy: 0.8798 - 653ms/epoch - 2ms/step

Test accuracy: 0.879800021648407
